# Flow 02_📍_BusLocator.py
I'm selecting route 16. Ballinteer - Dublin Airport towards Ballinter.

In [1]:
#Importing required libraries:
import os
import requests
import pandas as pd
import pytz  # For timezone handling
from datetime import datetime, timedelta
# API Key from environment variable
YOUR_API_KEY = os.environ.get("API_KEY")

1) Dropdown:Select a Route:

In [2]:
# Read the routes.txt file
file_path = "../data/Routes.txt" 
cr_df = pd.read_csv(file_path)
cr_df.head(20)

,route_id,incoming_bus,route_name
0,4486_88252,1,1. Sandymount - O'Connell Street - Santry
1,4486_88177,4,4. Monkstown Avenue - Harristown
2,4486_88174,6,6. Howth Dart Stn - Lower Abbey Street
3,4486_88218,7,7. Brides Glen Luas - Mountjoy Square
4,4486_88219,7A,7A. Loughlinstown - Mountjoy Sq
5,4486_88220,7B,7B. Shankill - Mountjoy Square
6,4486_88221,7D,7D. Dalkey - Mountjoy Square
7,4486_88222,7E,7E. Mountjoy Square - Dalkey
8,4486_88247,9,9. Limekiln Avenue - Charlestown
9,4492_88176,11,11. Phoenix Park - Sandyford Business District


In [3]:
route_name = '16. Ballinteer - Dublin Airport'
route_id = '4486_88181'

2) Dropdown: Towards

In [4]:
# Read the Towards.txt file
file_path = "../data/Towards.txt" 
sd_df = pd.read_csv(file_path)
sd_df.head(20)

,route_id,trip_id,trip_headsign,direction_id
0,4486_88174,4486_11484,Abbey St Lower,1
1,4486_88174,4486_11430,Howth Station,0
2,4486_88175,4492_604,Phoenix Pk,1
3,4486_88175,4492_567,Donnybrook Stadium,0
4,4486_88176,4486_563,Parnell Square,1
5,4486_88177,4492_8605,Heuston Station,1
6,4486_88177,4492_8414,Monkstown Ave,0
7,4486_88178,4486_1712,Grange Castle,0
8,4486_88178,4486_1919,Mountjoy Square,1
9,4486_88179,4492_2695,Eden Quay,1


Filtering by selected route 16. Ballinteer - Dublin Airport, below pd df would be the actual dropdown, creating variable trip_id

In [5]:
display = sd_df[(sd_df["route_id"] == route_id)]

display

,route_id,trip_id,trip_headsign,direction_id
15,4486_88181,4492_4685,Ballinteer,0
16,4486_88181,4492_4909,Dublin Airport,1
17,4486_88181,4486_4908,O'Connell Street,0
18,4486_88181,4492_5136,O'Connell Street,1


In [6]:
towards = '4492_4685'
trip_headsign = 'Ballinteer'

Filtering by the selected route and direction 16. Ballinteer - Dublin Airport towards Ballinteer

In [7]:
# Vehicles API Endpoint
VEHICLES_API_URL = "https://api.nationaltransport.ie/gtfsr/v2/Vehicles?format=json"

# Function to fetch Vehicles data with error handling
def fetch_vehicles_data():
    headers = {"x-api-key": YOUR_API_KEY}
    try:
        response = requests.get(VEHICLES_API_URL, headers=headers, verify=False)
        response.raise_for_status()  # Raise error for non-200 responses
        data = response.json()

        if "entity" not in data:
            print("⚠️ 'entity' key not found in Vehicles API response.")
            return None

        return data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP Error: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request Error: {req_err}")
    except ValueError as json_err:
        print(f"JSON Decode Error: {json_err}")

    return None

# Fetch Vehicles data
vehicles_data = fetch_vehicles_data()

if vehicles_data is None:
    print("❌ No data fetched. Please check the API key or network connection.")
else:
    print("✅ Vehicles data fetched successfully!")

# Function to process and filter Vehicles data
def process_vehicles_data(data, concat):
    vehicles = []

    for entity in data.get("entity", []):
        vehicle_info = entity.get("vehicle", {})
        trip_info = vehicle_info.get("trip", {})
        position_info = vehicle_info.get("position", {})

        # Extract trip details
        trip_id = trip_info.get("trip_id")
        route_id = trip_info.get("route_id")
        direction_id = trip_info.get("direction_id")
        start_time = trip_info.get("start_time")
        start_date = trip_info.get("start_date")
        schedule_relationship = trip_info.get("schedule_relationship")
        vehicle_id = vehicle_info.get("vehicle", {}).get("id")
        latitude = position_info.get("latitude")
        longitude = position_info.get("longitude")

        # Skip if not scheduled
        if schedule_relationship != "SCHEDULED":
            continue

        # Combine route_id and direction_id
        combined_id = str(route_id) + str(direction_id)

        # Filter by the provided 'concat' value
        if combined_id == concat:
            vehicles.append({
                "trip_id": trip_id,
                "route_id": route_id,
                "direction_id": direction_id,
                "vehicle_id": vehicle_id,
                "start_time": start_time,
                "start_date": start_date,
                "latitude": latitude,
                "longitude": longitude
            })

    return pd.DataFrame(vehicles)

# Example 'concat' value (update this to match your filter)
concat = '4486_881810'  # Example: route_id + direction_id

# Process and filter the data
if vehicles_data:
    filtered_vehicles = process_vehicles_data(vehicles_data, concat).sort_values(by="start_time", ascending=True).reset_index(drop=True)

    if filtered_vehicles.empty:
        print("⚠️ No vehicles found for the specified 'concat' filter.")
    else:
        print("✅ Filtered vehicles data:")

/home/joserico/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.nationaltransport.ie'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Vehicles data fetched successfully!
✅ Filtered vehicles data:


In [8]:
print("Buses operating route {} towards {}".format(route_name, trip_headsign))
filtered_vehicles

Buses operating route 16. Ballinteer - Dublin Airport towards Ballinteer


,trip_id,route_id,direction_id,vehicle_id,start_time,start_date,latitude,longitude
0,4492_4796,4486_88181,0,470,15:00:00,20250208,53.310635,-6.283067
1,4492_4799,4486_88181,0,640,15:12:00,20250208,53.292469,-6.282279
2,4492_4802,4486_88181,0,356,15:24:00,20250208,53.308407,-6.283988
3,4492_4805,4486_88181,0,716,15:36:00,20250208,53.335903,-6.265285
4,4492_4808,4486_88181,0,281,15:48:00,20250208,53.337872,-6.265918
5,4492_4810,4486_88181,0,1089,16:00:00,20250208,53.360783,-6.260209
6,4492_4813,4486_88181,0,5290,16:12:00,20250208,53.361912,-6.259232
7,4492_4819,4486_88181,0,913,16:36:00,20250208,53.407841,-6.237724
